# 1 basics
## 1.1 Tokenization
### 1.1.1 什么是Tokenization？
原始文本通常用 Unicode 字符串表示。语言模型会对一系列**词元**（tokens）（通常用整数索引表示）放置一个概率分布。因此，我们需要一种将字符串**编码**（encode）成词元的过程，同时需要一种将词元**解码**（decode）回字符串的过程。Tokenizer（分词器）就是一个实现了编码和解码方法的类。**词表大小**（vocabulary size）就是可能的词元（整数）总数。为了直观了解分词器是如何工作的，可以试用这个[交互式网站](https://tiktokenizer.vercel.app/?encoder=gpt2)。观察到的现象：
1. 一个单词与其前面的空格会被视为同一个 token（例如 " world"）。
2. 处在句首的单词和处在句中间的同一单词会被不同地表示（例如 "hello hello"）。
3. 数字会被分成每几位一个 token。
4. 以上的三条是课程里给的，第一条和第二条有点没看懂或者有问题（或者是我翻译的有问题）。
5. 同一个单词加空格和不加空格对应不同的 token，这个跟第二条说的有不一样的地方。测试的结果是课程中给的"hello hello"的例子中，两个之所以不一样，是因为第二个hello之前有空格，而第一个之前没有，当在第一个 hello 前加上空格后，两个对应的 token 是一样的。
6. 不同模型对应的 tokens 结果不同（如 gpt2 和 gpt4o）
7. 同一个单词大小写不同对应的 token 也不同（如 gpt2模型中 Science 为 5800，science 为 3783）
8. 同一个单词大小写不同的时候可能被划分为多个tokens（university 被划分为了两个部分）
9.  不同的模型对数字的划分风格不同，gpt4o 貌似只有三个数字划分这一种情况，gpt2 有两个数字划分在一起的情况
10. 换行符也有对应的 token，测试的 gpt2 和 gpt4o 的 token 都是198
![tiktoken](images\Tiktokenizer.png)

下图是 OpenAI 的 GPT-2 分词器（tiktoken）的实际运行示例。字符串 `Hello, 🌍! 你好!` 的 UTF-8 编码占用 20 字节。tokens 占 12 字节。$ratio_{compression} = 12 / 20 = 1.6666666667$。
![gpt2tiktoken](images\gpt2_tiktoken.png)

### 1.1.2 基于字符的分词器
**Unicode 字符串**是由 **Unicode 字符**组成的**序列**。每个字符可以通过 `ord` 转换成对应的码点（整数）。Unicode 总共有大约 15 万 个字符。
> **Unicode** 是一个全球统一的字符集，它为几乎所有语言的文字、符号、表情符号等分配了唯一的数字编号（叫码点，code point），这样计算机就能在全球范围内一致地存储和处理文本。**Unicode 字符串**就是一串这样的 Unicode 字符，每个字符在背后对应一个或多个码点。
> - Unicode 字符串不是直接用字节存的，而是一个字符序列，字符本身用 Unicode 编号来标识；等需要存储或传输时，可以用 UTF-8、UTF-16 等具体编码方式转成字节。

在例子 `Hello, 🌍! 你好!` 中，编码结果如下图所示。在 indices 中最大值为 127757 可以看出这个字符集是非常大的。
![charactertokenizer](images\charactertokenizer.png)
问题 1：这个字符集非常大。
问题 2：许多字符（例如 🌍）非常少见，这会导致词表的使用效率很低。

### 1.1.3 基于字节的分词器
Unicode 字符串可以表示为一系列字节，而字节又可以用 0 到 255 之间的整数表示。最常用的 Unicode 编码是 UTF-8。有些 Unicode 字符只需要一个字节表示，有些则需要多个字节。
![bytetokenizer](images\bytetokenizer.png)
使用基于字节的分词器优点是词表很小：一个字节最多表示 256 个值。但压缩率（compression ratio）非常差，也就是说分词后的序列会很长。由于 Transformer 的上下文长度有限（注意力计算是平方级增长的），这会导致效率很低。

### 1.1.4 基于单词的分词器
另一种方法是把字符串按单词拆分，这种方法更接近传统自然语言处理的做法。

```python
string = "I'll say supercalifragilisticexpialidocious!"
segments = regex.findall(r"\w+|.", string) 
```
这个正则表达式会把所有字母数字字符连续地当作一个整体保留下来，而不会把它们拆开。就像下面这样：
![wordtokenizer](images\wordtokenization.png){width="800px" height="px"}

要把它变成一个 Tokenizer，需要把这些片段（segments）映射成整数 ID，然后建立一个片段 → 整数的映射表。但是这种方法有几个问题：
1. 单词的数量非常庞大（就像 Unicode 字符一样）。
2. 很多单词非常罕见，模型学不到太多关于它们的东西。
3. 不容易得到固定大小的词表（vocabulary size）。
$ Size_{vocabulary} = Number_{distinct ~ segments ~ of ~ training ~ data}$
1. 对于训练中没见过的新单词，要用特殊的 UNK token 来代替，这会让结果变得很糟糕，还可能影响困惑度（perplexity）的计算。

### 1.1.5 基于BPE的分词器
**字节对编码**（Byte Pair Encoding，**BPE**）最早由 Philip Gage 于 1994 年提出，用于数据压缩。它后来被改进并应用到自然语言处理（NLP）中的神经机器翻译任务中。 [Sennrich 等，2015] 在此之前，研究论文通常使用基于单词的分词方法。BPE 随后被 GPT-2 采用。 [Radford 等，2019] GPT-2 论文中先用基于单词的分词方法将文本分成初始片段，再对每个片段运行原始的 BPE 算法。
**基本思想**：在原始文本上训练分词器，让它自动确定词表。常见的字符序列用单个 token 表示，稀有的字符序列则用多个 token 表示。**流程**：先把每个字节当作一个 token，然后不断合并出现频率最高的一对相邻 token。
1. 把字符串转成 UTF-8 编码的字节序列
2. 统计所有相邻 token 对的出现次数，找到出现次数最多的一对 token
3. 创建新 token，把新 token 对应的字节序列放进词表
4. 重复步骤 2、步骤 3， 共 `num_merges` 次

![bpe_steps](images\bpe_steps.jpg)